# Основы анализа данных в Python

*Алла Тамбовцева*

## Практикум 5. Проверка статистических гипотез


Импортируем необходимые библиотеки:
    
* `numpy`: для работы с массивами;
* модуль `stats` из `scipy` для статистических вычислений.

In [1]:
import numpy as np
from scipy import stats as st

### Задача 1

Дан массив `votes` из 0 и 1, где 1 соответствуют респондентам, которые планируют голосовать за Корнелиуса Фаджа на выборах президента Волшебного Клуба Веселых и Находчивых (после смещения с поста министра у него возможностей не на кнат ведро). 

In [2]:
votes = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
                  0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
                  0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0])

Проверьте, можно ли считать долю собирающихся голосовать за Фаджа равной 0.6, приняв уровень значимости равным 5%.

1. Сформулируйте нулевую гипотезу и двустороннюю альтернативу.
2. Примените подходящий критерий (тест) в Python и приведите код с выдачей результатов.
3. Сделайте статистический и содержательный выводы.

$$
H_0: p = 0.6
$$
$$
H_1: p \ne 0.6
$$

Так как проверяем гипотезу о доле, нам может пригодиться биномиальный критерий (*binomial test*) и z-критерий для долей (*proportion z-test)*. Первый критерий особенно актуален для работы с маленькими выборками, второй – для работы с большими выборками. Это неслучайно, ведь по сути мы изначально работаем не с долями, а с числом успехов – числом случаев, которые нас интересуют, число успехов в серии испытаний Бернулли описывается биномиальным распределением, а оно, в свою очередь, при большом числе испытаний стремится к нормальному.

Для проверки гипотезы с помощью биномиального критерия нам понадобится функция `binom_test()`. На вход этой функции нужно подать число успехов `x` (в нашем примере – число единиц), общее число испытаний `n` и значение доли из нулевой гипотезы `p`, по умолчанию алгоритм предполагает двустороннюю альтернативу:

In [3]:
# число единиц = сумма бинарного массива

st.binom_test(x = votes.sum(), n = votes.size, p = 0.6)

0.0007498240919579229

Эта функция возвращает только p-value. P-value в данном случае меньше уровня значимости 0.05, поэтому на имеющихся данных есть основания нулевую гипотезу отвергнуть в пользу альтернативы. Доля собирающихся голосовать за Фаджа не равна 0.6. 

Если бы нам нужно было проверять нулевую гипотезу против односторонней альтернативы, мы бы выбрали ее направление, исходя из данных. Посчитаем долю единиц по выборке:

In [4]:
votes.mean()

0.36

Выборочная доля меньше заявленного значения 0.6 в $H_0$, поэтому альтернатива будет левосторонней:

$$
H_0: p = 0.6
$$
$$
H_1: p < 0.6
$$

Учтем это при проверке:

In [5]:
# значения alternative: two-sided, less, greater

st.binom_test(x = votes.sum(), n = votes.size, p = 0.6, alternative = "less")

0.000519301231366599

Небольшая деталь, которая может ввести в заблуждение: в отличие от z-критерия, статистика которого имеет стандартное нормальное распределение, этот критерий подразумевает использование биномиального распределения, которое не всегда симметрично (симметрично только при $p=0.5$), поэтому p-value в случае проверки односторонней альтернативы необязательно в два раза меньше p-value в случае проверки двусторонней альтернативы. Можете изменить значение `p` в функции на 0.5 и убедиться, что при проверке гипотезы $p=0.5$ будет работать привычная нам закономерность: p-value в случае двусторонней альтернативы в два раза больше p-value в случае односторонней альтернативы.

**Дополнительно.** Более знакомый нам с ТВиМС z-критерий для проверки гипотезы в Python тоже есть. Проще всего его найти в другом модуле `stats` внутри библиотеки `statsmodels`:

In [6]:
from statsmodels.stats.proportion import proportions_ztest

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


На вход функция `proportions_ztest()` принимает следующие аргументы:

* `count`: число успехов (число единиц в нашем случае);
* `nobs`: общее число наблюдений;
* `value`: значение доли из нулевой гипотезы;
* `alternative`: тип альтернативы, по умолчанию двусторонняя ("two-sided"), можно изменить на левостороннюю ("smaller") или правосторонюю ("larger").

Применим ее для нашего случая, сначала для двусторонней альтернативы:

In [7]:
proportions_ztest(count = votes.sum(), nobs = votes.size, value = 0.6)

(-3.5355339059327378, 0.0004069520174449578)

Функция возвращает два значения – наблюдаемое значение статистики ($z_{набл}$) и p-value. Вывод все тот же: нулевую гипотезу о равенстве доли 0.6 стоит отвергнуть. Посмотрим, что произойдет с p-value, если альтернатива будет односторонней:

In [8]:
proportions_ztest(count = votes.sum(), nobs = votes.size, value = 0.6, alternative = "smaller")

(-3.5355339059327378, 0.0002034760087224789)

Вот тут с p-value все привычно, как только выбрали одностороннюю альтернтиву, p-value сократилось ровно в два раза (вспоминаем графики с закрашенными площадями и определение p-value):

*Двусторонняя альтернатива*

$$
\text{p-value} = \text{P}(z < -3.535) + \text{P}(z > 3.535) \approx 0.0004
$$

*Односторонняя альтернатива*

$$
\text{p-value} = \text{P}(z < -3.535) \approx 0.0002
$$

### Задача 2

Дан массив `Ron` – число шоколадных лягушек, съеденных Роном в течение октября (одно наблюдение – один день месяца).

In [9]:
Ron = np.array([8., 7., 7., 6., 9., 7., 7., 7., 6., 8., 8., 8., 7., 6., 8., 6., 6.,
       7., 9., 7., 6., 7., 8., 8., 7., 7., 8., 7., 6., 5., 8.])

Проверьте, можно ли считать среднее число лягушек, съедаемых Роном за месяц, равным 5, приняв уровень значимости равным 1%.

1. Сформулируйте нулевую гипотезу и одностороннюю альтернативу, направление выберите, исходя из данных.

2. Примените подходящий критерий (тест) в Python и приведите код с выдачей результатов.

3. Сделайте статистический и содержательный выводы.

Смотрим на выборочное среднее, чтобы выбрать тип альтернативы:

In [10]:
Ron.mean()

7.129032258064516

Среднее по выборке больше 5, поэтому:
    
$$
H_0: \mu = 5
$$

$$
H_1: \mu > 5
$$

Для проверки гипотезы о равенстве среднего числу используется одновыборочный критерий Стьюдента, одновыборочный t-test. Реализуется он с помощью функции `ttest_1samp()` из `stats`, которая принимает на вход выборку (список, массив, столбец датафрейма) и значение среднего из гипотезы `popmean`:

In [11]:
# ttest_1samp: t-test for one sample

st.ttest_1samp(Ron, popmean = 5)

Ttest_1sampResult(statistic=12.384679313173228, pvalue=2.5261785289223924e-13)

Функция возвращает наблюдаемое значение статистики критерия ($t_{набл}$) и p-value. Эта функция не предполагает выбора альтернативной гипотезы, p-value вычисляется, исходя из двусторонней альтернативы. То есть, в нашем случае, формально мы должны разделить полученное p-value на 2. Но оно здесь очень мало, $2.52 \times 10^{-13}$, практически 0, поэтому существенной разницы не будет, нулевая гипотеза отвергается на любом конвенциональном уровне значимости, среднее число лягушек, поглощаемых Роном, больше 5.

### Задача 3

Даны два массива значений – число шоколадных лягушек, съеденных Гарри и Роном в течение октября (одно наблюдение – один день месяца). Проверьте, можно ли считать средние значения числа лягушек, съеденных Гарри и Роном одинаковыми.

In [ ]:
Harry = np.array([3., 3., 2., 3., 3., 2., 2., 4., 3., 2., 2., 4., 2., 3., 3., 3., 3.,
       4., 2., 3., 3., 4., 2., 4., 4., 5., 3., 2., 3., 2., 2.])
Ron = np.array([8., 7., 7., 6., 9., 7., 7., 7., 6., 8., 8., 8., 7., 6., 8., 6., 6.,
       7., 9., 7., 6., 7., 8., 8., 7., 7., 8., 7., 6., 5., 8.])

1. Сформулируйте нулевую гипотезу и одностороннюю альтернативу (направление выберите, исходя из данных).
2. Примените подходящий критерий (тест) в Python и приведите код с выдачей результатов.
3. Сделайте статистический и содержательный выводы.

In [ ]:
### YOUR CODE HERE ###

### Задача 4 

*Из независимого экзамена*

Ниже приведены данные об уровнях осадков в двух различных регионах России, измеренные за одинаковые промежутки времени. 
Предполагая, что все необходимые предпосылки выполнены, дисперсии генеральных совокупностей равны, а выборки независимы, проверьте гипотезу о равенстве средних уровней осадков ($𝐻_0:\mu_1=\mu_2$) при помощи t-теста на уровне значимости 5%. Выберите верное утверждение.

Выберите один ответ:

* Нулевая гипотеза отвергается.
* Нулевая гипотеза не отвергается.


Регион 1: [103.01, 101.99, 105.21, 106.80, 112.70, 106.13, 110.48, 109.26, 100.44, 100.28].

Регион 2: [107.38, 106.31, 106.00, 105.27, 105.27, 104.66, 103.70, 105.07, 105.12, 104.74].

In [ ]:
### YOUR CODE HERE ###